In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import statsmodels.api as sm
import torch
from torch import nn
from tqdm.notebook import tqdm
from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

current_dir = Path.cwd()
import sys
import gc

sys.path.append(str(current_dir.parent))
from utils import get_competition_data_path, submit
%matplotlib inline

In [ ]:
path_dict = get_competition_data_path("m5-forecasting-accuracy")
competition_path = path_dict.get("competition_path")
train_path = path_dict.get("train_path")
submission_path = path_dict.get("sample_submission_path")
calendar_path = competition_path / "calendar.csv"
sell_prices_path = competition_path / "sell_prices.csv"

In [ ]:
plt.rcParams["figure.figsize"] = 14, 6
sns.set()

In [ ]:
def reduce_mem_usage(props, skip=None):
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of properties dataframe is :", start_mem_usg, " MB")
    NAlist = []  # Keeps track of columns that have missing values filled in.
    for col in props.columns:
        if skip is not None and col in skip:
            continue
        if props[col].dtype != object:  # Exclude strings

            # Print current column type
            print("******************************")
            print("Column: ", col)
            print("dtype before: ", props[col].dtype)

            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()

            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(mn - 1, inplace=True)

            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = props[col] - asint
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)

            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)

            # Print new column type
            print("dtype after: ", props[col].dtype)
            print("******************************")

    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage is: ", mem_usg, " MB")
    print("This is ", 100 * mem_usg / start_mem_usg, "% of the initial size")
    return props, NAlist

## Objectives
The objective of the M5 forecasting competition is to advance the theory and practice of forecasting by identifying the method(s) that provide the most accurate point forecasts for each of the 42,840 time series of the competition. I addition, to elicit information to estimate the uncertainty distribution of the realized values of these series as precisely as possible. 
To that end, the participants of M5 are asked to provide 28 days ahead point forecasts (PFs) for all the series of the competition, as well as the corresponding median and 50%, 67%, 95%, and 99% prediction intervals (PIs).
The M5 differs from the previous four ones in five important ways, some of them suggested by the discussants of the M4  competition, as follows:
- First, it uses grouped unit sales data, starting at the product-store level and being aggregated to that of product departments, product categories, stores, and three geographical areas: the States of California (CA), Texas (TX), and Wisconsin (WI).
- Second, besides the time series data, it includes explanatory variables such as sell prices, promotions, days of the week, and special events (e.g. Super Bowl, Valentine’s Day, and Orthodox Easter) that typically affect unit sales and could improve forecasting accuracy.
- Third, in addition to point forecasts, it assesses the distribution of uncertainty, as the participants are asked to provide information on nine indicative quantiles.
- Fourth, instead of having a single competition to estimate both the point forecasts and the uncertainty distribution, there will be two parallel tracks using the same dataset, the first requiring 28 days ahead point forecasts and the second 28 days ahead probabilistic forecasts for the median and four prediction intervals (50%, 67%, 95%, and 99%).
- Fifth, for the first time it focuses on series that display intermittency, i.e., sporadic demand including zeros.

## Datasets info
The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

sns.set()
plt.rcParams["figure.figsize"] = 20, 10

img = mpimg.imread("../data/m5-forecasting-accuracy/m5-forecasting-hierarchical.png")
imgplot = plt.imshow(img)
plt.grid(False)
plt.axis("off")
plt.show()

Remove outlier days: Xmas

In [ ]:
plt.rcParams["figure.figsize"] = 14, 6

Model choice
- I decide to turn the time series problem into a supervised learning problem
- Reasons:
    - There's no existing package (that I know of) for time series analysis and forecast capable of dealing of this much amount of time series (statsmodels can only fit one series at a time)
    - It's more straight forward to incorporate external features 
    - Random Forests and LightGBM are powerful algorithm that do really well in many types of problem, including time series forecast

## Model 1

- No feature engineering
- Use existing data
- Drop features that I consider insignificant: event, price

In [ ]:
calendar = pd.read_csv(calendar_path)
calendar = calendar.drop(columns=['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'])

In [ ]:
encoder = OrdinalEncoder(dtype='int')

# Convert event to number
all_event_names = np.append(calendar['event_name_1'].dropna().unique(), calendar['event_name_2'].dropna().unique()).reshape(-1, 1)
all_event_types = np.append(calendar['event_type_1'].dropna().unique(), calendar['event_type_2'].dropna().unique()).reshape(-1, 1)

name_codes = encoder.fit_transform(all_event_names) 
name_codes_dict = dict(zip(all_event_names.flatten().tolist(), name_codes.flatten().tolist()))

type_codes = encoder.fit_transform(all_event_types) 
type_codes_dict = dict(zip(all_event_types.flatten().tolist(), type_codes.flatten().tolist()))

# Change d to number
d = calendar['d'].str.split('_', expand=True)[1].tolist()

# Get quarter
quarter = pd.to_datetime(calendar['date']).dt.quarter


calendar = calendar.assign(d=d, quarter=quarter, event_name_1=event_name_1, event_type_1=event_type_1, event_name_2=event_name_2, event_type_2=event_type_2)
calendar.head()

Sales:
- Changes days of sales to number
- Turn to long format

In [ ]:
sales = pd.read_csv(train_path)
sample_submission = pd.read_csv(submission_path)

outliers = ['d_331', 'd_697', 'd_1062', 'd_1427', 'd_1792']
sales = sales.loc[:, ~sales.columns.isin(outliers)]
sales.head()

In [ ]:
sales_cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [ ]:
subset = 800

In [ ]:
# Turn d into number
d = sales.columns[6:].to_series().str.split('_', expand=True)[1].tolist()
sales.columns = sales.columns[:6].tolist() + d

# Keep only necessary columns
sales = sales.drop(columns=['id'])

# Use a subset of d 
if subset:
    d = d[-subset:]
sales = sales.reindex(columns=sales_cat_features + d)

In [ ]:
# Encode cat features to number
item_id = encoder.fit_transform(sales[['item_id']]).astype(int)
dept_id = encoder.fit_transform(sales[['dept_id']]).astype(int)
cat_id = encoder.fit_transform(sales[['cat_id']]).astype(int)
store_id = encoder.fit_transform(sales[['store_id']]).astype(int)
state_id = encoder.fit_transform(sales[['state_id']]).astype(int)

# Get item dict to map back later
item_id_dict = dict(zip(sales['item_id'], item_id.flatten()))
dept_id_dict = dict(zip(sales['dept_id'], dept_id.flatten()))
cat_id_dict = dict(zip(sales['cat_id'], cat_id.flatten()))
store_id_dict = dict(zip(sales['store_id'], store_id.flatten()))

In [ ]:
# Assign number values
sales = sales.assign(item_id=item_id, dept_id=dept_id, cat_id=cat_id, store_id=store_id, state_id=state_id)

# To long format
sales = sales.melt(id_vars=sales_cat_features, var_name='d', value_name='sale')

sales.head()

In [ ]:
gc.collect()

Attach calendar and prices to sales

In [ ]:
# Merge with calendar
sales = sales.merge(calendar, on='d')

# Drop unused columns
sales = sales.drop(columns=['wm_yr_wk', 'weekday', 'date'])
sales.head()

In [ ]:
del calendar, sell_prices
gc.collect()

In [ ]:
sales, _ = reduce_mem_usage(sales)

In [ ]:
sales.to_feather('sales_1.feather')

del sales
gc.collect()

## Model 2

- Add features, specifically lagged values of sales
    - Lag 1
    - Lag 7
    - Lag 28
    - Lag 365

In [ ]:
sales = pd.read_feather('sales_1.feather')

In [ ]:
sales.head()

In [ ]:
group = sales.groupby(['item_id', 'store_id'])['sale']

# Assign lag values
sales['lag1'] = group.shift(1)
sales['lag7'] = group.shift(7)
sales['lag28'] = group.shift(28)
sales['lag365'] = group.shift(365)

# Test if the assignments work as intended
test_item_id = 1437
test_store_id = 0
test_sale = sales.query('item_id == @test_item_id and store_id == @test_store_id')
test_sale = test_sale.sort_values('d')
np.testing.assert_allclose(test_sale['sale'].shift(7).to_numpy(), test_sale['lag7'].to_numpy())

del group, test_item_id, test_store_id, test_sale

sales = sales.dropna().reset_index(drop=True)

gc.collect()


In [ ]:
sales.to_feather('sales_2.feather')

del sales
gc.collect()

## Model 3

- Add more sales features: moving averages of lag. Idea: not only the last seasonal values affecting sales, but also a week and 4 weeks
    - moving average of 7 for lag 1
    - moving average of 28 for lag 1
    - moving average of 7 for lag 7
    - moving average of 28 for lag 7
    - moving average of 7 for lag 28
    - moving average of 28 for lag 28

In [ ]:
sales = pd.read_feather('sales_2.feather')

In [ ]:
sales.head()

In [ ]:
group_lag1 = sales.groupby(['item_id', 'store_id'])['lag1']
group_lag7 = sales.groupby(['item_id', 'store_id'])['lag7']
group_lag28 = sales.groupby(['item_id', 'store_id'])['lag28']

# Assign ma values
sales['ma7_lag1'] = group_lag1.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag1'] = group_lag1.transform(lambda x: x.rolling(28).mean())
sales['ma7_lag7'] = group_lag7.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag7'] = group_lag7.transform(lambda x: x.rolling(28).mean())
sales['ma7_lag28'] = group_lag28.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag28'] = group_lag28.transform(lambda x: x.rolling(28).mean())

# Test if the assignments work as intended
test_item_id = 1437
test_store_id = 0
test_sale = sales.query('item_id == @test_item_id and store_id == @test_store_id')
test_sale = test_sale.sort_values('d')
np.testing.assert_allclose(test_sale['lag7'].rolling(7).mean().to_numpy(), test_sale['ma7_lag7'].to_numpy())

del group_lag7, group_lag28

sales = sales.dropna().reset_index(drop=True)

gc.collect()


In [ ]:
sales.to_feather('sales_3.feather')

del sales
gc.collect()

## Model 4

#### All features
- All the features above
- Add event features
    - event_name
    - event_type
- Add price features
    - price
    - changes in price
    - mean of changes in price
    - std of changes in price

In [ ]:
calendar = pd.read_csv(calendar_path)

In [ ]:
encoder = OrdinalEncoder(dtype='int')

# Convert event to number
all_event_names = np.append(calendar['event_name_1'].dropna().unique(), calendar['event_name_2'].dropna().unique()).reshape(-1, 1)
all_event_types = np.append(calendar['event_type_1'].dropna().unique(), calendar['event_type_2'].dropna().unique()).reshape(-1, 1)

name_codes = encoder.fit_transform(all_event_names) 
name_codes_dict = dict(zip(all_event_names.flatten().tolist(), name_codes.flatten().tolist()))

type_codes = encoder.fit_transform(all_event_types) 
type_codes_dict = dict(zip(all_event_types.flatten().tolist(), type_codes.flatten().tolist()))

event_name_1 = calendar['event_name_1'].map(name_codes_dict).fillna(-1).astype('int')
event_name_2 = calendar['event_name_2'].map(name_codes_dict).fillna(-1).astype('int')

event_type_1 = calendar['event_type_1'].map(type_codes_dict).fillna(-1).astype('int')
event_type_2 = calendar['event_type_2'].map(type_codes_dict).fillna(-1).astype('int')

# Change d to number
d = calendar['d'].str.split('_', expand=True)[1].tolist()

# Get quarter
quarter = pd.to_datetime(calendar['date']).dt.quarter


calendar = calendar.assign(d=d, quarter=quarter, event_name_1=event_name_1, event_type_1=event_type_1, event_name_2=event_name_2, event_type_2=event_type_2)
calendar.head()

In [ ]:
sales = pd.read_csv(train_path)
sample_submission = pd.read_csv(submission_path)

outliers = ['d_331', 'd_697', 'd_1062', 'd_1427', 'd_1792']
sales = sales.loc[:, ~sales.columns.isin(outliers)]
sales.head()

In [ ]:
sales_cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [ ]:
subset = 800

In [ ]:
# Turn d into number
d = sales.columns[6:].to_series().str.split('_', expand=True)[1].tolist()
sales.columns = sales.columns[:6].tolist() + d

# Keep only necessary columns
sales = sales.drop(columns=['id'])

# Use a subset of d 
if subset:
    d = d[-subset:]
sales = sales.reindex(columns=sales_cat_features + d)

In [ ]:
# Encode cat features to number
item_id = encoder.fit_transform(sales[['item_id']]).astype(int)
dept_id = encoder.fit_transform(sales[['dept_id']]).astype(int)
cat_id = encoder.fit_transform(sales[['cat_id']]).astype(int)
store_id = encoder.fit_transform(sales[['store_id']]).astype(int)
state_id = encoder.fit_transform(sales[['state_id']]).astype(int)

# Get item dict to map back later
item_id_dict = dict(zip(sales['item_id'], item_id.flatten()))
dept_id_dict = dict(zip(sales['dept_id'], dept_id.flatten()))
cat_id_dict = dict(zip(sales['cat_id'], cat_id.flatten()))
store_id_dict = dict(zip(sales['store_id'], store_id.flatten()))

In [ ]:
# Assign number values
sales = sales.assign(item_id=item_id, dept_id=dept_id, cat_id=cat_id, store_id=store_id, state_id=state_id)

# To long format
sales = sales.melt(id_vars=sales_cat_features, var_name='d', value_name='sale')

sales.head()

In [ ]:
gc.collect()

Attach days to price and convert categorical columns to number

In [ ]:
sell_prices = pd.read_csv(sell_prices_path)
sell_prices = sell_prices.assign(item_id=sell_prices['item_id'].map(item_id_dict), store_id=sell_prices['store_id'].map(store_id_dict))

# Sort to make sure that diff is correct
sell_prices = sell_prices.sort_values(['item_id', 'store_id', 'wm_yr_wk'])

# Take diff to get change in price
sell_prices['price_change'] = sell_prices.groupby(['item_id', 'store_id'])['sell_price'].diff().fillna(0)

# Get mean and std of change in price
group = sell_prices.groupby(['item_id', 'store_id'])['price_change']
sell_prices['price_change_mean'] = group.transform(lambda x: x.mean())
sell_prices['price_change_std'] = group.transform(lambda x: x.std())


sell_prices = sell_prices.merge(calendar[['wm_yr_wk', 'd']], how='left').drop(columns='wm_yr_wk')

del group
gc.collect()

sell_prices.head()

Attach calendar and prices to sales

In [ ]:
# Merge with price
sales = sales.merge(sell_prices, on=['d', 'item_id', 'store_id'], how='inner')

# Merge with calendar
sales = sales.merge(calendar, on='d')

# Drop unused columns
sales = sales.drop(columns=['wm_yr_wk', 'weekday', 'date'])
sales.head()

In [ ]:
del calendar, sell_prices
gc.collect()

In [ ]:
sales, _ = reduce_mem_usage(sales)

In [ ]:
group = sales.groupby(['item_id', 'store_id'])['sale']

# Assign lag values
sales['lag1'] = group.shift(1)
sales['lag7'] = group.shift(7)
sales['lag28'] = group.shift(28)
sales['lag365'] = group.shift(365)


group_lag1 = sales.groupby(['item_id', 'store_id'])['lag1']
group_lag7 = sales.groupby(['item_id', 'store_id'])['lag7']
group_lag28 = sales.groupby(['item_id', 'store_id'])['lag28']

# Assign ma values
sales['ma7_lag1'] = group_lag1.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag1'] = group_lag1.transform(lambda x: x.rolling(28).mean())
sales['ma7_lag1'] = group_lag7.transform(lambda x: x.rolling(7).mean())
sales['ma7_lag7'] = group_lag7.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag7'] = group_lag7.transform(lambda x: x.rolling(28).mean())
sales['ma7_lag28'] = group_lag28.transform(lambda x: x.rolling(7).mean())
sales['ma28_lag28'] = group_lag28.transform(lambda x: x.rolling(28).mean())

sales = sales.dropna().reset_index(drop=True)

del group, group_lag1, group_lag7, group_lag28
gc.collect()

In [ ]:
sales.to_feather('sales_4.feather')

del sales
gc.collect()